In [ ]:
#기본 설치
!git clone https://github.com/clovaai/dmfont.git
%cd dmfont

Cloning into 'dmfont'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 83 (delta 12), reused 6 (delta 6), pack-reused 61 (from 1)
Receiving objects: 100% (83/83), 192.23 KiB | 974.00 KiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/dmfont


In [ ]:
!pip install torch torchvision pillow PyYAML tqdm scikit-image imageio matplotlib fire fonttools tensorboardX sconf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.8 MB/s eta 0:00:00
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p checkpoints/
!cp "/content/drive/Shareddrives/2025 인공프/dmfont/korean-handwriting.pth" checkpoints/

!mkdir -p data/charset/
!cp "/content/drive/Shareddrives/2025 인공프/dmfont/korean11172.txt" data/charset

!mkdir -p style
!cp "/content/drive/Shareddrives/2025 인공프/dmfont/Styleimg.pdf" style

import os
os.makedirs("configs", exist_ok=True)

In [ ]:
!apt-get install poppler-utils  # pdf2image에서 필요
!pip install pdf2image
from pdf2image import convert_from_path
import os

pdf_files = [
    "/content/dmfont/style/Styleimg.pdf",
]

output_dir = "/content/dmfont/style"
os.makedirs(output_dir, exist_ok=True)

for pdf_path in pdf_files:
    images = convert_from_path(pdf_path, dpi=300)
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]

    if len(images) > 1:
        # 여러 페이지면 _p1, _p2 등 붙임
        for i, img in enumerate(images):
            out_path = f"{output_dir}/{base_name}_p{i+1}.png"
            img.save(out_path)
            print(f"✅ 저장 완료: {out_path}")
    else:
        # 한 페이지만 있으면 기본 이름으로 저장
        img = images[0]
        out_path = f"{output_dir}/{base_name}.png"
        img.save(out_path)
        print(f"✅ 저장 완료: {out_path}")

from PIL import Image, ImageChops
import os

def trim_whitespace(path):
    img = Image.open(path)
    bg = Image.new(img.mode, img.size, img.getpixel((0, 0)))
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()
    if bbox:
        trimmed = img.crop(bbox)
        trimmed.save(path)  # 덮어쓰기
        print(f"✅ 여백 제거 완료: {path}")
    else:
        print(f"⚠ 여백 없음: {path}")

# data/raw 안의 모든 이미지 여백 제거
for fname in os.listdir("/content/dmfont/style"):
    if fname.lower().endswith((".png", ".jpg", ".jpeg")):
        trim_whitespace(f"/content/dmfont/style/{fname}")

from PIL import Image
import os

raw_dir = "/content/dmfont/style"

for fname in os.listdir(raw_dir):
    if fname.lower().endswith((".png", ".jpg", ".jpeg")):
        path = os.path.join(raw_dir, fname)
        img = Image.open(path)
        img.save(path, dpi=(300, 300))  # 해상도만 300dpi로 설정
        print(f"✅ 해상도 300dpi 저장 완료: {fname}")

crop_code = '''
import os
from PIL import Image, ImageChops, ImageOps
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--src_dir", required=True)
parser.add_argument("--dst_dir", required=True)
args = parser.parse_args()

# 각 템플릿별 행, 열
layout = {
    "Styleimg.png": (4, 7)
}

pad = 20
os.makedirs(args.dst_dir, exist_ok=True)

start_unicode = 0xAC00
count = 0

skip_indices = {
    "Styleimg.png": set()  # 건너뛸 칸이 없으면 빈 set()
}

for fname in sorted(os.listdir(args.src_dir)):
    if not fname.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    if fname not in layout:
        print(f"⚠ {fname}의 행/열 정보가 layout에 없습니다. 건너뜀.")
        continue

    rows, cols = layout[fname]
    skip_set = skip_indices.get(fname, set())

    img_path = os.path.join(args.src_dir, fname)
    img = Image.open(img_path).convert("L")

    w, h = img.size
    cell_w = w // cols
    cell_h = h // rows

    index = 1  # 1부터 시작

    for r in range(rows):
        for c in range(cols):
            if index in skip_set:
                print(f"⏩ {fname} - 칸 {index} 건너뜀")
                index += 1
                continue

            left = c * cell_w + pad
            upper = r * cell_h + pad + 40
            right = (c + 1) * cell_w - pad
            lower = (r + 1) * cell_h - pad
            cropped = img.crop((left, upper, right, lower))

            bg = Image.new("L", cropped.size, 255)
            diff = ImageChops.difference(cropped, bg)
            bbox = diff.getbbox()

            if bbox:
                cropped = cropped.crop(bbox)

            padding = 30
            cropped = ImageOps.expand(cropped, border=padding, fill=255)

            # 정사각형 배경 만들고 중앙 배치
            max_side = max(cropped.size)
            square = Image.new("L", (max_side, max_side), 255)
            offset = ((max_side - cropped.width) // 2, (max_side - cropped.height) // 2)
            square.paste(cropped, offset)
            cropped = square

            # 최종 리사이즈
            cropped = cropped.resize((512, 512), Image.BICUBIC)

            fname_out = f"uni{start_unicode + count:04X}.png"
            cropped.save(os.path.join(args.dst_dir, fname_out))
            count += 1
            index += 1

print(f"✅ 총 {count}개 글자 이미지 저장 완료: {args.dst_dir}")
'''

with open("/content/dmfont/style/crop.py", "w") as f:
    f.write(crop_code)

print("✅ crop.py 생성 완료")

!python style/crop.py \
  --src_dir=style \
  --dst_dir=style/cropped


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
✅ 저장 완료: /content/dmfont/style/Styleimg.png
✅ 여백 제거 완료: /content/dmfont/style/Styleimg.png
✅ 해상도 300dpi 저장 완료: Styleimg.png
✅ crop.py 생성 완료
✅ 총 28개 글자 이미지 저장 완료: style/cropped


In [ ]:
from PIL import Image
import os
import numpy as np

src_folder = "style/cropped"
dst_folder = "style_cleaned"
os.makedirs(dst_folder, exist_ok=True)

for fname in os.listdir(src_folder):
    if fname.lower().endswith(".png"):
        # 이미지 열기 및 흑백화
        img = Image.open(os.path.join(src_folder, fname)).convert("L")

        # NumPy 배열로 변환
        img_np = np.array(img)

        # threshold 적용 (0 또는 255로 변환)
        threshold = 200
        img_bin = np.where(img_np > threshold, 255, 0).astype(np.uint8)

        # 다시 이미지로 변환
        img_cleaned = Image.fromarray(img_bin, mode="L")

        # 리사이즈
        img_cleaned = img_cleaned.resize((128, 128), Image.Resampling.LANCZOS)

        # 저장
        img_cleaned.save(os.path.join(dst_folder, fname))


In [ ]:
import yaml

cfg = {
    'style_imgs': [
        'style_cleaned/uniAC00.png',
        'style_cleaned/uniAC01.png',
        'style_cleaned/uniAC02.png',
        'style_cleaned/uniAC03.png',
        'style_cleaned/uniAC04.png',
        'style_cleaned/uniAC05.png',
        'style_cleaned/uniAC06.png',
        'style_cleaned/uniAC07.png',
        'style_cleaned/uniAC08.png',
        'style_cleaned/uniAC09.png',
        'style_cleaned/uniAC0A.png',
        'style_cleaned/uniAC0B.png',
        'style_cleaned/uniAC0C.png',
        'style_cleaned/uniAC0D.png',
        'style_cleaned/uniAC0E.png',
        'style_cleaned/uniAC0F.png',
        'style_cleaned/uniAC10.png',
        'style_cleaned/uniAC11.png',
        'style_cleaned/uniAC12.png',
        'style_cleaned/uniAC13.png',
        'style_cleaned/uniAC14.png',
        'style_cleaned/uniAC15.png',
        'style_cleaned/uniAC16.png',
        'style_cleaned/uniAC17.png',
        'style_cleaned/uniAC18.png',
        'style_cleaned/uniAC19.png',
        'style_cleaned/uniAC1A.png',
        'style_cleaned/uniAC1B.png'
    ],
    'style_chars': ['각','깪','냓','댼','떥','렎','멷','볠','뽉','솲','쐛','욄','죭','쭖','춣','퀨','튑','퓺','흣','읬','잉','잊','잋','잌','잍','잎','잏','이'],

    'charset_path': 'data/charset/korean11172.txt',
    'output_dir': 'output',
    'checkpoint': 'checkpoints/korean-handwriting.pth',
    'num_font_samples': 1,
    'target_chars': ['가','나','다','라','마','바','사','아','자','차','카','타','파','하','\n','안','녕','하','세','요','\n','감','사','합','니','다','\n','구','름','빵',' ','먹','고','싶','다','\n','오','늘',' ','저','녁','은',' ','고','추','잡','채','밥'],

    'C': 32,
    'n_comps': 68,
    'n_comp_types': 3,
    'language': 'kor'
}

with open('configs/infer_colab.yaml', 'w') as f:
    yaml.dump(cfg, f, allow_unicode=True)


In [ ]:
### class StyleImageDataset 정의

import os

code = """
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from . import kor_decompose

class StyleImageDataset(Dataset):
    def __init__(self, style_imgs, style_chars, language='kor', transform=None, style_id=0):
        assert len(style_imgs) == len(style_chars), "style_imgs와 style_chars 길이는 같아야 합니다."
        self.style_imgs = style_imgs
        self.style_chars = style_chars
        self.language = language
        self.transform = transform
        self.style_id = style_id

        if language == 'kor':
            self.decompose = kor_decompose.decompose
        else:
            raise NotImplementedError(f"Language {language}는 지원되지 않습니다.")

    def __len__(self):
        return len(self.style_imgs)

    def __getitem__(self, idx):
        path = self.style_imgs[idx]
        char = self.style_chars[idx]
        comp_ids = self.decompose(char)

        img = Image.open(path).convert('L')  # 흑백
        if self.transform:
            img = self.transform(img)

        return (
            self.style_id,
            torch.as_tensor(comp_ids),
            img
        )
"""

os.makedirs("dmfont/datasets", exist_ok=True)
with open("datasets/style_image_dataset.py", "w") as f:
    f.write(code.strip())



In [ ]:

####inference.py 수정

# 덮어쓸 내용 정의 (여러 줄 문자열)
new_code = '''

"""
DMFont
Copyright (c) 2020-present NAVER Corp.
MIT license
"""
import torch
from torch.utils.data import DataLoader
from datasets import get_ma_val_dataset
from datasets.nonpaired_dataset import EncodeDataset, DecodeDataset
import os

os.chdir("/content/dmfont")  # or wherever your 'style_cleaned' 폴더가 있는 최상위 경로



def get_val_loader(data, fonts, chars, style_avails, transform, content_font, language,
                   B=32, n_max_match=3, n_workers=2):
    val_dset, collate_fn = get_ma_val_dataset(
        data, fonts, chars, style_avails, n_max_match, transform=transform,
        content_font=content_font, language=language
    )
    loader = DataLoader(val_dset, batch_size=B, shuffle=False,
                        num_workers=n_workers, collate_fn=collate_fn)

    return loader


def infer_2stage(gen, encode_loader, decode_loader, reset_memory=True):
    """ 2-stage infer; encode first, decode second """
    # stage 1. encode
    if reset_memory:
        gen.reset_dynamic_memory()

    for style_ids, style_comp_ids, style_imgs in encode_loader:
        style_ids = style_ids.to("cpu")
        style_comp_ids = style_comp_ids.to("cpu")
        style_imgs = style_imgs.to("cpu")

        gen.encode_write(style_ids, style_comp_ids, style_imgs, reset_dynamic_memory=False)

    # stage 2. decode
    outs = []
    for trg_ids, trg_comp_ids in decode_loader:
        trg_ids = trg_ids.to("cpu")
        trg_comp_ids = trg_comp_ids.to("cpu")
        # print("🧠 target_style_ids:", style_ids)


        out = gen.read_decode(trg_ids, trg_comp_ids)

        outs.append(out.detach().cpu())

    return torch.cat(outs)


def get_val_decode_loader(chars, language, B=32, num_workers=2, style_id=0):
    decode_dset = DecodeDataset(chars, language=language, style_id=style_id)
    loader = DataLoader(decode_dset, batch_size=B, shuffle=False, num_workers=num_workers)

    return loader



###추가
from datasets.style_image_dataset import StyleImageDataset  # 직접 만든 Dataset일 수도 있음
from torch.utils.data import DataLoader

def get_styleimg_encode_loader(style_imgs, style_chars, language, transform, style_id=0, B=32, num_workers=2):
    dset = StyleImageDataset(style_imgs, style_chars, language=language, transform=transform, style_id=style_id)
    return DataLoader(dset, batch_size=B, shuffle=False, num_workers=num_workers)



from torchvision import transforms

def get_transform():
    return transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

#수정후
from PIL import Image
import numpy as np
import torch
import os

def save_sentence_image(
    imgs, path, text=None, target_height=128,
    max_side_crop1=25, max_side_crop2=25,
    overlap_margin=0, space_margin=30, line_spacing=20
):
    imgs = imgs.squeeze(1)  # [N, 1, H, W] → [N, H, W]
    imgs = imgs.mul(0.5).add(0.5).clamp(0, 1)  # [-1,1] → [0,1]
    imgs = imgs.mul(255).byte().cpu().numpy()  # [0,255]

    # 문자 이미지 전처리
    char_imgs = []
    for img in imgs:
        pil_img = Image.fromarray(img, mode='L')
        arr = np.array(pil_img)

        col_sum = np.mean(arr, axis=0)
        non_white = np.where(col_sum < 245)[0]
        if len(non_white) > 0:
            x0 = max(non_white[0] - max_side_crop1, 0)
            x1 = min(non_white[-1] + max_side_crop2 + 1, arr.shape[1])
            cropped = pil_img.crop((x0, 0, x1, arr.shape[0]))
        else:
            cropped = pil_img

        # 높이 맞춤
        W, H = cropped.size
        if H < target_height:
            top_pad = (target_height - H) // 2
            padded = Image.new('L', (W, target_height), 255)
            padded.paste(cropped, (0, top_pad))
        else:
            padded = cropped.resize((W, target_height), Image.Resampling.LANCZOS)

        char_imgs.append(padded)

    # 총 이미지 크기 계산
    max_line_width = 0
    line_width = 0
    img_idx = 0
    line_count = 1
    for c in text:
        if c == '\\n':
            max_line_width = max(max_line_width, line_width)
            line_width = 0
            line_count += 1
        elif c == ' ':
            line_width += space_margin
        else:
            if img_idx < len(char_imgs):
                line_width += char_imgs[img_idx].size[0]
                if line_width > 0:
                    line_width -= overlap_margin
                img_idx += 1
    max_line_width = max(max_line_width, line_width)
    total_height = line_count * target_height + (line_count - 1) * line_spacing

    # 이미지 생성 및 붙이기
    final_img = Image.new('L', (max_line_width, total_height), 255)
    x_offset = 0
    y_offset = 0
    img_idx = 0

    for c in text:
        if c == '\\n':
            y_offset += target_height + line_spacing
            x_offset = 0
        elif c == ' ':
            x_offset += space_margin
        else:
            if img_idx >= len(char_imgs):
                break
            im = char_imgs[img_idx]
            final_img.paste(im, (x_offset, y_offset))
            x_offset += im.size[0] - overlap_margin
            img_idx += 1

    os.makedirs(os.path.dirname(path), exist_ok=True)
    final_img.save(path)


def main(config, checkpoint, save_dir):
    import os
    import torch
    import yaml
    with open(config, 'r') as f:
      cfg = yaml.safe_load(f)
    from models.comp_encoder import ComponentEncoder
    from models.decoder import Decoder
    from models.ma_core import MACore
    # from datasets.transforms import get_transform
    # from utils.misc import save_imgs
    from inference import infer_2stage, get_val_decode_loader

    os.makedirs(save_dir, exist_ok=True)
    device = torch.device("cpu")

    with open(config, 'r') as f:
      cfg = yaml.safe_load(f)


    # style-aware feed-forward block 설정
    sa = {'n_heads': 4}  # Clova 기본 세팅


    # 최종 Generator 생성
    gen = MACore(
        C_in=1,
        C=32,              # ← from config
        C_out=1,
        comp_enc={
            'norm': 'none',
            'activ': 'relu',
            'pad_type': 'zero',
            'sa': {
                'C_qk_ratio': 0.5,
                'n_heads': 2,
                'area': False,
                'ffn_mult': 2
            }
        },
        dec={
            'norm': 'IN',
            'activ': 'relu',
            'pad_type': 'zero'
        },
        n_comps=68,
        n_comp_types=3,
        language='kor'
    ).to(device)




    # 사전학습된 파라미터 로드
    ckpt = torch.load(checkpoint, map_location="cpu")
    gen.load_state_dict(ckpt["generator_ema"])  # ✅ 이거로!

    gen.eval()

    # 인코딩 및 디코딩용 데이터로더
    from inference import get_styleimg_encode_loader  # 위에서 만든 함수

    encode_loader = get_styleimg_encode_loader(
      cfg['style_imgs'], cfg['style_chars'], cfg['language'],
      transform=get_transform(), style_id=0
    )
    import re
    valid_target_chars = [c for c in cfg['target_chars'] if re.match(r'[가-힣]', c)]
    decode_loader = get_val_decode_loader(valid_target_chars, cfg['language'], style_id=0)
    text = ''.join(cfg['target_chars'])  # 출력용 문장은 그대로 유지


    # 추론 실행
    with torch.no_grad():
        outs = infer_2stage(gen, encode_loader, decode_loader)

    # save_sentence_image_with_padding(outs, os.path.join(save_dir, "sentence.png"))
    save_sentence_image(outs, os.path.join(save_dir, "sentence.png"), text=text, overlap_margin=0, space_margin=50)



if __name__ == '__main__':
    import fire
    fire.Fire(main)

'''

# 경로에 덮어쓰기
with open('inference.py', 'w', encoding='utf-8') as f:
    f.write(new_code)


In [ ]:
!pip install fire
!python inference.py \
    --config configs/infer_colab.yaml \
    --checkpoint checkpoints/korean-handwriting.pth \
    --save_dir output/

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
